In [1]:
import pandas as pd
import numpy as np

from src.language_processing2 import processing_text
from src.language_processing2 import vectorize_text
from src.language_processing2 import vectorize_desc
from src.modeling import test_model
from src.modeling import test_model2
from src.modeling import test_model3
from src.modeling import test_model4

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split,KFold

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [3]:
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,1000)

In [4]:
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,1000)

In [5]:
df.head()

,text,spam,spam_marketing,spam_hijack,spam_corporate,spam_bot,spam_known,spam_own,Docusign,onespan,...,desc-TF-IDF-990,desc-TF-IDF-991,desc-TF-IDF-992,desc-TF-IDF-993,desc-TF-IDF-994,desc-TF-IDF-995,desc-TF-IDF-996,desc-TF-IDF-997,desc-TF-IDF-998,desc-TF-IDF-999
0,just closed a deal in 27 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,just closed a deal in 2 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,just closed a deal in 2 hours using #AdobeSign...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,just closed a deal in 26 hours using #AdobeSig...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,just closed a deal in 6 days using #AdobeSign ...,0,0,0,0,1,0,0,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
test_model3(df,'spam_hijack',RandomForestClassifier,col_name_lst1+col_name_lst2)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9201875000000002

Log-Loss score of this model is

0.25771578635491316

AUC score of this model is

0.8878695993945074



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9156244843571472

Log-Loss score of this model is

0.23974509975605765

AUC score of this model is

0.8888592257717483


In [3]:
df2 = df.sample(frac=1)
df2_training = df2[:16000]
df2_testing = df2[16000:]
y = df2_training['spam_marketing']
X = df2_training[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count','favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
model = RandomForestClassifier(n_estimators=250)
kf = KFold(n_splits=5, shuffle=True)

ll_performance = []
auc_performance = []
acc_performance = []
#kfold split on X_ (which is X_train of len 110)
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    y_pred = model.predict_proba(X_test)
    log_ll = log_loss(y_test, y_pred)
    ll_performance.append(log_ll)
    auc = roc_auc_score(y_test,y_predict)
    auc_performance.append(auc)
    acc = accuracy_score(y_test,y_predict)
    acc_performance.append(acc)
        
        
print("\n" + "Score summary for initial test (first 80% of data)" + "\n")
print("\n" + "Accuracy score of this model is" + "\n")
print(np.mean(acc_performance))
print("\n" + "Log-Loss score of this model is" + "\n")
print(np.mean(ll_performance))
print("\n" + "AUC score of this model is" + "\n")
print(np.mean(auc_performance))
print("\n")

y = df2_testing['spam_marketing']
X = df2_testing[['Docusign', 'onespan', 'signnow','adobe sign','listed_count', 'statuses_count','followers_count','favourites_count', 'friends_count','time_float_sin','time_float_cos', 'is_description_none']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    #model = model()
kf = KFold(n_splits=5, shuffle=True)

ll_performance = []
auc_performance = []
acc_performance = []
#kfold split on X_ (which is X_train of len 110)
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    y_pred = model.predict_proba(X_test)
    log_ll = log_loss(y_test, y_pred)
    ll_performance.append(log_ll)
    auc = roc_auc_score(y_test,y_predict)
    auc_performance.append(auc)
    acc = accuracy_score(y_test,y_predict)
    acc_performance.append(acc)
        
        
print("\n" + "Score summary for final test (last 20% of data)" + "\n")
print("\n" + "Accuracy score of this model is" + "\n")
print(np.mean(acc_performance))
print("\n" + "Log-Loss score of this model is" + "\n")
print(np.mean(ll_performance))
print("\n" + "AUC score of this model is" + "\n")
print(np.mean(auc_performance))


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9475625000000001

Log-Loss score of this model is

0.16399420401484804

AUC score of this model is

0.643786813679372



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9435842354164526

Log-Loss score of this model is

0.17401056828825015

AUC score of this model is

0.603851441111064


In [4]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,1000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,1000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9566250000000001

Log-Loss score of this model is

0.11927033054210238

AUC score of this model is

0.6945236159589906



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9555698768061122

Log-Loss score of this model is

0.1254935648403675

AUC score of this model is

0.6451012199577858


In [5]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,2000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,1000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9570624999999999

Log-Loss score of this model is

0.10531826178893973

AUC score of this model is

0.6923168101650614



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9500748752338879

Log-Loss score of this model is

0.12830144448145822

AUC score of this model is

0.6273866424361378


In [6]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,1000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,2000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.955875

Log-Loss score of this model is

0.11426385733827857

AUC score of this model is

0.6977616236084914



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9515736252377793

Log-Loss score of this model is

0.12368892743877087

AUC score of this model is

0.6291551757113043


In [7]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,2000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,2000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9578749999999999

Log-Loss score of this model is

0.10821673114173773

AUC score of this model is

0.7037926113725805



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9500720732500832

Log-Loss score of this model is

0.1321088562327118

AUC score of this model is

0.6315221541210816


In [8]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,3000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,2000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.958

Log-Loss score of this model is

0.11015041798686931

AUC score of this model is

0.6963508690491242



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9503267424447619

Log-Loss score of this model is

0.13143805894453783

AUC score of this model is

0.6411372088577394


In [9]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,2000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,3000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9567499999999999

Log-Loss score of this model is

0.11941480718359825

AUC score of this model is

0.6906272861704869



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9505776756610347

Log-Loss score of this model is

0.12354345730597116

AUC score of this model is

0.6323403621732464


In [10]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,3000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,3000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9579374999999999

Log-Loss score of this model is

0.11194482903300354

AUC score of this model is

0.700034479689825



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9505742510141625

Log-Loss score of this model is

0.1413341921910043

AUC score of this model is

0.619154548023811


In [11]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,4000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,3000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9563750000000001

Log-Loss score of this model is

0.10888142393933811

AUC score of this model is

0.6991485538069215



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9523195755928532

Log-Loss score of this model is

0.12946088931375027

AUC score of this model is

0.5976114271609482


In [12]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,3000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,4000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9563124999999999

Log-Loss score of this model is

0.11780373235295535

AUC score of this model is

0.6927245563698061



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9518226904648491

Log-Loss score of this model is

0.12466953900829361

AUC score of this model is

0.6128748593817976


In [13]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,4000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,4000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9570000000000001

Log-Loss score of this model is

0.10993216614789499

AUC score of this model is

0.6958403033288428



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9480779947758569

Log-Loss score of this model is

0.13841091722215978

AUC score of this model is

0.6128184910300932


In [14]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,5000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,4000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)

KeyboardInterrupt: 

In [ ]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,4000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,5000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)

In [15]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,5000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,5000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9567499999999999

Log-Loss score of this model is

0.11553369705850762

AUC score of this model is

0.6895808656089613



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.947330487763114

Log-Loss score of this model is

0.1373477982510662

AUC score of this model is

0.6336540909167466


In [ ]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,6000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,5000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)

In [ ]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,5000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,6000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)

In [16]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,6000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,6000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9572499999999999

Log-Loss score of this model is

0.11425174293262566

AUC score of this model is

0.6973548485450924



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9498264326698859

Log-Loss score of this model is

0.12321365281611592

AUC score of this model is

0.6051245671585574


In [ ]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,7000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,6000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)

In [ ]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,6000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,7000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)

In [17]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,7000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,7000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9569375000000001

Log-Loss score of this model is

0.11460957651030898

AUC score of this model is

0.6888547974079732



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9478292408803212

Log-Loss score of this model is

0.14488685372547305

AUC score of this model is

0.6209239975307546


In [ ]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,8000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,8000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)


Score summary for initial test (first 80% of data)


Accuracy score of this model is

0.9570624999999999

Log-Loss score of this model is

0.11812024392888434

AUC score of this model is

0.6915929767009258



Score summary for final test (last 20% of data)


Accuracy score of this model is

0.9463270662295571

Log-Loss score of this model is

0.1424789466251267

AUC score of this model is

0.6010916837343231


In [ ]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,9000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,9000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)

In [ ]:
df = pd.read_excel('testing2.xlsx')
df.drop('Unnamed: 0', axis=1, inplace=True)
processing_text(df,'text')
col_name_lst1 = vectorize_text(df,10000)
processing_text(df,'description')
col_name_lst2 = vectorize_desc(df,10000)
test_model4(df,'spam_marketing',col_name_lst1+col_name_lst2)